In [90]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [91]:
team = pd.read_csv("whitewater.csv")
opponents = pd.read_csv("whitewater_opponents.csv")

In [92]:
team['Opponent'] = team.Opponent.apply(lambda i : i.replace("vs\xa0", "").replace("at\xa0", ""))

advanced = team.merge(opponents, on="Opponent", suffixes=("", " OPP"))

In [93]:
advanced['Margin'] = advanced.Score.apply(lambda i : int(i.split("-")[0]) - int(i.split("-")[1]))

advanced['FGM'] = advanced['FGM/A'].apply(lambda i : float(i.split("-")[0]))
advanced['FGA'] = advanced['FGM/A'].apply(lambda i : float(i.split("-")[1]))
advanced['3PM'] = advanced['3FG/A'].apply(lambda i : float(i.split("-")[0]))
advanced['3PA'] = advanced['3FG/A'].apply(lambda i : float(i.split("-")[1]))
advanced['FTA'] = advanced['FTM/A'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP FGM'] = advanced['FGM/A OPP'].apply(lambda i : float(i.split("-")[0]))
advanced['OPP FGA'] = advanced['FGM/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP 3PM'] = advanced['3FG/A OPP'].apply(lambda i : float(i.split("-")[0]))
advanced['OPP 3PA'] = advanced['3FG/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP FTA'] = advanced['FTM/A OPP'].apply(lambda i : float(i.split("-")[1]))

advanced.loc[len(advanced)] = advanced.sum(numeric_only=True, axis=0)

advanced['POSS'] = round(0.96 * (advanced.FGA + 
                                 advanced.TO + 
                                 0.44 * advanced.FTA) - 
                                 advanced.OFF, 1)
advanced['PPP'] = round(advanced.PTS / advanced.POSS, 2)
advanced['REB'] = advanced.TOT.astype(int)
advanced['DRB%'] = 100 * round(advanced.DEF / (advanced.DEF + advanced['OFF OPP']), 3)
advanced['ORB%'] = 100 * round(advanced.OFF / (advanced.OFF + advanced['DEF OPP']), 3)
advanced['RB%'] = 100 * round(advanced.TOT / (advanced.TOT + advanced['TOT OPP']), 3)
advanced['TO%'] = 100 * round(advanced.TO / advanced.POSS, 3)

advanced['OPP POSS'] = round(0.96 * (advanced['OPP FGA'] + 
                                     advanced['TO OPP'] + 
                                     0.44 * advanced['OPP FTA'] - 
                                     advanced['OFF OPP']), 1)
advanced['OPP PPP'] = round(advanced['PTS OPP'] / advanced['OPP POSS'], 2)
advanced['OPP REB'] = advanced['TOT OPP'].astype(int)

advanced.loc[len(advanced)-1, 'Opponent'] = "Total"
advanced.loc[len(advanced)-1, 'W/L'] = ""
advanced.loc[len(advanced)-1, 'POSS'] /= len(advanced)-1
advanced.loc[len(advanced)-1, 'OPP POSS'] /= len(advanced)-1

advanced['POSS'] = advanced['POSS'].round(1)
advanced['OPP POSS'] = advanced['OPP POSS'].round(1)
advanced['Margin'] = advanced['Margin'].astype(int)

advanced = advanced.set_index('Opponent')

In [94]:
advanced[['W/L', 'Margin',
          'POSS', 'OPP POSS', 
          'PPP', 'OPP PPP', 
          'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 
          'TO%']].to_csv("ww_adv.csv")

In [113]:
advanced[['W/L', 'Margin', '3FG/A', 'PCT.1']].sort_values(by=['PCT.1'])


,W/L,Margin,3FG/A,PCT.1
Opponent,,,,
Carroll (WI),W,4,2-15,0.133
Ill. Wesleyan,W,9,3-17,0.176
Ripon,W,14,3-13,0.231
Edgewood,W,12,6-23,0.261
UW-River Falls,L,-3,5-19,0.263
Calvin,L,-2,4-12,0.333
Millikin,W,10,9-19,0.474
Wis.-La Crosse,W,5,8-16,0.500
Augustana (IL),W,28,12-22,0.545


In [100]:
advanced[['W/L', 'Margin', '3FG/A OPP', 'PCT.1 OPP']].sort_values(by='PCT.1 OPP')

,W/L,Margin,3FG/A OPP,PCT.1 OPP
Opponent,,,,
Wis.-La Crosse,W,5,1-14,0.071
Carroll (WI),W,4,4-17,0.235
Ill. Wesleyan,W,9,4-12,0.333
Augustana (IL),W,28,4-11,0.364
Ripon,W,14,6-16,0.375
Millikin,W,10,5-11,0.455
Edgewood,W,12,6-13,0.462
UW-River Falls,L,-3,8-17,0.471
Calvin,L,-2,5-10,0.500


In [117]:
advanced[['W/L', 'Margin', 'ORB%', 'TO', 'TO OPP', 'PCT.1']].sort_values(by='PCT.1')


,W/L,Margin,ORB%,TO,TO OPP,PCT.1
Opponent,,,,,,
Carroll (WI),W,4,42.5,23.0,23.0,0.133
Ill. Wesleyan,W,9,30.6,16.0,25.0,0.176
Ripon,W,14,28.9,17.0,26.0,0.231
Edgewood,W,12,46.0,12.0,19.0,0.261
UW-River Falls,L,-3,30.3,18.0,20.0,0.263
Calvin,L,-2,26.2,14.0,25.0,0.333
Millikin,W,10,54.3,23.0,23.0,0.474
Wis.-La Crosse,W,5,33.3,21.0,11.0,0.500
Augustana (IL),W,28,38.7,20.0,21.0,0.545


In [120]:
advanced[['W/L', 'Margin', 'ORB%', 'TO', 'PCT.1', 'TO OPP', '3FG/A OPP']].sort_values(by='PCT.1')


,W/L,Margin,ORB%,TO,PCT.1,TO OPP,3FG/A OPP
Opponent,,,,,,,
Carroll (WI),W,4,42.5,23.0,0.133,23.0,4-17
Ill. Wesleyan,W,9,30.6,16.0,0.176,25.0,4-12
Ripon,W,14,28.9,17.0,0.231,26.0,6-16
Edgewood,W,12,46.0,12.0,0.261,19.0,6-13
UW-River Falls,L,-3,30.3,18.0,0.263,20.0,8-17
Calvin,L,-2,26.2,14.0,0.333,25.0,5-10
Millikin,W,10,54.3,23.0,0.474,23.0,5-11
Wis.-La Crosse,W,5,33.3,21.0,0.500,11.0,1-14
Augustana (IL),W,28,38.7,20.0,0.545,21.0,4-11


In [98]:
advanced

,Date,Score,W/L,FGM/A,PCT,3FG/A,PCT.1,FTM/A,PCT.2,OFF,DEF,TOT,AVG,PF,AST,TO,BLK,STL,PTS,AVG.1,Date OPP,Score OPP,W/L OPP,FGM/A OPP,PCT OPP,3FG/A OPP,PCT.1 OPP,FTM/A OPP,PCT.2 OPP,OFF OPP,DEF OPP,TOT OPP,AVG OPP,PF OPP,AST OPP,TO OPP,BLK OPP,STL OPP,PTS OPP,AVG.1 OPP,Margin,FGM,FGA,3PM,3PA,FTA,OPP FGM,OPP FGA,OPP 3PM,OPP 3PA,OPP FTA,POSS,PPP,REB,DRB%,ORB%,RB%,TO%,OPP POSS,OPP PPP,OPP REB
Opponent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Carroll (WI),11/11/22,56-52,W,19-55,0.345,2-15,0.133,16-24,0.667,17.0,26.0,43.0,43.0,16.0,10.0,23.0,5.0,13.0,56.0,56.0,11/11/22,52-56,L,22-59,0.373,4-17,0.235,4-8,0.500,13.0,23.0,36.0,36.0,23.0,11.0,23.0,2.0,10.0,52.0,52.0,4,19.0,55.0,2.0,15.0,24.0,22.0,59.0,4.0,17.0,8.0,68.0,0.82,43,66.7,42.5,54.4,33.8,69.6,0.75,36
Millikin,11/12/22,79-69,W,29-63,0.460,9-19,0.474,12-17,0.706,19.0,21.0,40.0,41.5,18.0,19.0,23.0,5.0,12.0,79.0,67.5,11/12/22,69-79,L,26-59,0.441,5-11,0.455,12-17,0.706,14.0,16.0,30.0,33.0,19.0,11.0,23.0,4.0,17.0,69.0,60.5,10,29.0,63.0,9.0,19.0,17.0,26.0,59.0,5.0,11.0,17.0,70.7,1.12,40,60.0,54.3,57.1,32.5,72.5,0.95,30
Ill. Wesleyan,11/16/22,70-61,W,26-62,0.419,3-17,0.176,15-16,0.938,11.0,21.0,32.0,38.3,18.0,14.0,16.0,1.0,14.0,70.0,68.3,11/16/22,61-70,L,24-54,0.444,4-12,0.333,9-12,0.750,10.0,25.0,35.0,33.7,18.0,7.0,25.0,3.0,5.0,61.0,60.7,9,26.0,62.0,3.0,17.0,16.0,24.0,54.0,4.0,12.0,12.0,70.6,0.99,32,67.7,30.6,47.8,22.7,71.3,0.86,35
Ripon,11/19/22,67-53,W,23-58,0.397,3-13,0.231,18-22,0.818,11.0,24.0,35.0,37.5,16.0,12.0,17.0,8.0,14.0,67.0,68.0,11/19/22,53-67,L,19-53,0.358,6-16,0.375,9-12,0.750,8.0,27.0,35.0,34.0,14.0,11.0,26.0,1.0,9.0,53.0,58.8,14,23.0,58.0,3.0,13.0,22.0,19.0,53.0,6.0,16.0,12.0,70.3,0.95,35,75.0,28.9,50.0,24.2,73.2,0.72,35
Edgewood,11/22/22,69-57,W,28-75,0.373,6-23,0.261,7-13,0.538,23.0,24.0,47.0,39.4,16.0,17.0,12.0,7.0,9.0,69.0,68.2,11/22/22,57-69,L,19-51,0.373,6-13,0.462,13-16,0.813,9.0,27.0,36.0,34.4,16.0,13.0,19.0,2.0,2.0,57.0,58.4,12,28.0,75.0,6.0,23.0,13.0,19.0,51.0,6.0,13.0,16.0,66.0,1.05,47,72.7,46.0,56.6,18.2,65.3,0.87,36
Calvin,11/26/22,64-66,L,25-64,0.391,4-12,0.333,10-19,0.526,11.0,19.0,30.0,37.8,18.0,6.0,14.0,4.0,14.0,64.0,67.5,11/26/22,66-64,W,22-45,0.489,5-10,0.500,17-21,0.810,6.0,31.0,37.0,34.8,19.0,15.0,25.0,7.0,5.0,66.0,59.7,-2,25.0,64.0,4.0,12.0,19.0,22.0,45.0,5.0,10.0,21.0,71.9,0.89,30,76.0,26.2,44.8,19.5,70.3,0.94,37
Wis.-La Crosse,11/30/22,58-53,W,22-53,0.415,8-16,0.500,6-12,0.500,11.0,33.0,44.0,38.7,21.0,10.0,21.0,6.0,5.0,58.0,66.1,11/30/22,53-58,L,17-59,0.288,1-14,0.071,18-24,0.750,11.0,22.0,33.0,34.6,11.0,5.0,11.0,5.0,10.0,53.0,58.7,5,22.0,53.0,8.0,16.0,12.0,17.0,59.0,1.0,14.0,24.0,65.1,0.89,44,75.0,33.3,57.1,32.3,66.8,0.79,33
UW-River Falls,12/03/22,64-67,L,22-53,0.415,5-19,0.263,15-21,0.714,10.0,22.0,32.0,37.9,18.0,11.0,18.0,3.0,9.0,64.0,65.9,12/03/22,67-64,W,23-52,0.442,8-17,0.471,13-18,0.722,10.0,23.0,33.0,34.4,17.0,11.0,20.0,1.0,5.0,67.0,59.8,-3,22.0,53.0,5.0,19.0,21.0,23.0,52.0,8.0,17.0,18.0,67.0,0.96,32,68.8,30.3,49.2,26.9,67.1,1.00,33
Augustana (IL),12/07/22,92-64,W,30-56,0.536,12-22,0.545,20-26,0.769,12.0,23.0,35.0,37.6,23.0,18.0,20.0,4.0,15.0,92.0,68.8,12/07/22,64-92,L,21-53,0.396,4-11,0.364,18-24,0.750,9.0,19.0,28.0,33.7,22.0,6.0,21.0,0.0,10.0,64.0,60.2,28,30.0,56.0,12.0,22.0,26.0,21.0,53.0,4.0,11.0,24.0,71.9,1.28,35,71.9,38.7,55.6,27.8,72.5,0.88,28
